In [1]:

%pip install --upgrade firebase-admin


Requirement already up-to-date: firebase-admin in /opt/conda/lib/python3.7/site-packages (4.3.0)
Note: you may need to restart the kernel to use updated packages.


In [2]:

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore



In [3]:
# Use the application default credentials
cred = credentials.ApplicationDefault()
firebase_admin.initialize_app(cred, {
  'projectId': 'tldr-278619',
})

db = firestore.client()



In [22]:
def is_good_tldr(doc_id, doc_dict):
    if doc_dict.get('skip_reason') == 'p':
        print(f'Document {doc_id} should be published')
        return True
    
    doc_mark_for_publish = doc_dict.get('published', False)
   
    return doc_mark_for_publish

def output_doc(writer, doc):
    doc_id = doc.id
    doc_dict = doc.to_dict()
#     print(f'{doc_id} => {doc_dict}')
    if not doc_dict:
        return

    csv_row = {
        'id': doc_id,
        'title': doc_dict['title'],
        'summary': doc_dict['summary'],
        'good_tldr': is_good_tldr(doc_id, doc_dict)
    }
#     print(csv_row)
    writer.writerow(csv_row)

In [23]:
docs = db.collection(u'urls').stream()

In [24]:
import itertools
# first10 = itertools.islice(docs, 300)

import logging
import os

import csv


docs_count = 0

with open('local-summary-training-data.csv', 'w', newline='') as csvfile:
    fieldnames = ['id', 'title', 'summary', 'good_tldr']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()

    for doc in docs:
        output_doc(writer, doc)
        docs_count += 1
        
print(f'Processed {docs_count} documents')
        
        


Document https___techcrunch.com_2020_07_07_nvidias-ampere-gpus-come-to-google-cloud_amp_ should be published
Document https___www.engadget.com_apple-music-beats-1-renamed-130259193.html should be published
Document https___www.engadget.com_reddit-linkedin-ios-14-clipboard-snooping-fix-152930127.html should be published
Document https___www.engadget.com_uber-buys-postmates-124051166.html should be published
Document https___www.marketwatch.com_amp_story_amazon-apple-facebook-and-google-ceos-get-grilled-by-house-members-in-historic-antitrust-hearing-2020-07-29 should be published
Document https___www.pcmag.com_news_report-apple-to-launch-apple-one-subscription-bundles should be published
Document https___www.theguardian.com_technology_2020_aug_17_google-giving-user-data-authorities-documents-reveal should be published
Document https___www.theverge.com_2020_7_10_21319784_ios-apps-crashing-spotify-tiktok-pinterest-tinder-facebook-sdk-certification-issue should be published
Document https__

In [69]:
with open('documents.csv', 'r') as lines:
    for line in itertools.islice(lines, 20):
        print(line)


id,title,summary,should_publish

http___techcrunch.com_2020_07_02_festos-latest-biomimetic-robots-are-a-flying-feathered-bird-and-ball-bottomed-helper-arm_,Festo’s latest biomimetic robots are a flying feathered bird and ball-bottomed helper arm – TechCrunch,"You could be excused for thinking that German robotics company Festo does nothing but put together fabulous prototype robots built to resemble kangaroos, jellyfish, and other living things. They do in fact actually make real industrial robots, but it’s hard not to marvel at their biomimetic experiments; Case in point, the feathered BionicSwift and absurd BionicMobileAssistant motile arm. Of course, it’s silly to criticize such a machine, which is aspirational rather than practical.",False

http___techcrunch.com_2020_07_02_growth-capital-investor-kennet-raises-250m-fund-backed-by-edmond-de-rothschild_,"Growth capital investor Kennet raises $250M fund, backed by Edmond de Rothschild – TechCrunch","Venture capital is “not the only fr

In [74]:
from google.cloud import storage
    
# https://console.cloud.google.com/storage/browser/[bucket-id]/
client = storage.Client(project='tldr-news-discovery')
bucket = client.get_bucket('tldr-training-dataset')
blob = bucket.blob('summary-training-data.csv')
blob.upload_from_filename('local-summary-training-data.csv')